Tee Trains

In [1]:
!pip install mcroute

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import sys
import math
import pandas as pd
import altair as alt
import networkx as nx
import numpy as np

sys.path.append(r"..")

from mcroute import Network, StateSpace
import mcroute.matrix as matrix
import mcroute.vector as vector


In [3]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [4]:
%cd /gdrive/My\ Drive/TravelTimes_2022

/gdrive/.shortcut-targets-by-id/1ZtYrt91AlM2bxT-ESLEdQDoPCrHaeb9j/TravelTimes_2022


In [5]:
Stops = pd.read_csv('stops_w_id.csv')

In [6]:
Stops['stop_name'] = Stops['stop_name'].astype(str)

In [7]:
Open = pd.read_csv('Open.csv')
Closed = pd.read_csv('Closed.csv')

In [8]:
Orange_Open = Open.loc[Open['route_id']=='Orange']
Green_B_Open = Open.loc[(Open['route_id']=='Green-B')]
Green_C_Open = Open.loc[(Open['route_id']=='Green-C')]
Green_D_Open = Open.loc[(Open['route_id']=='Green-D')]
Green_E_Open = Open.loc[(Open['route_id']=='Green-E')]
Red_Open = Open.loc[Open['route_id']=='Red']

In [9]:
Orange_Closed = Closed.loc[Closed['route_id']=='Orange']
Green_B_Closed = Closed.loc[(Closed['route_id']=='Green-B')]
Green_C_Closed = Closed.loc[(Closed['route_id']=='Green-C')]
Green_D_Closed = Closed.loc[(Closed['route_id']=='Green-D')]
Green_E_Closed = Closed.loc[(Closed['route_id']=='Green-E')]
Red_Closed = Closed.loc[Closed['route_id']=='Red']

Splitting by time period

In [10]:
#Orange

#Early AM Hours
Orange_Early_AM_Open = Orange_Open.loc[(Orange_Open['start_time_sec'] >= 10800) & (Orange_Open['start_time_sec'] < 25200)]
Orange_Early_AM_Closed = Orange_Closed.loc[(Orange_Closed['start_time_sec'] >= 10800) & (Orange_Closed['start_time_sec'] < 25200)]

#AM Peak Hours
Orange_AM_Peak_Open = Orange_Open.loc[(Orange_Open['start_time_sec'] >= 25200) & (Orange_Open['start_time_sec'] < 32400)]
Orange_AM_Peak_Closed = Orange_Closed.loc[(Orange_Closed['start_time_sec'] >= 25200) & (Orange_Closed['start_time_sec'] < 32400)]

#Midday Hours
Orange_Midday_Open = Orange_Open.loc[(Orange_Open['start_time_sec'] >= 32400) & (Orange_Open['start_time_sec'] < 57600)]
Orange_Midday_Closed = Orange_Closed.loc[(Orange_Closed['start_time_sec'] >= 32400) & (Orange_Closed['start_time_sec'] < 57600)]

#PM Peak Hours
Orange_PM_Peak_Open = Orange_Open.loc[(Orange_Open['start_time_sec'] >= 57600) & (Orange_Open['start_time_sec'] < 66600)]
Orange_PM_Peak_Closed = Orange_Closed.loc[(Orange_Closed['start_time_sec'] >= 57600) & (Orange_Closed['start_time_sec'] < 66600)]

#Evening Hours
Orange_Evening_Open = Orange_Open.loc[(Orange_Open['start_time_sec'] >= 66600) | (Orange_Open['start_time_sec'] < 10800)]
Orange_Evening_Closed = Orange_Closed.loc[(Orange_Closed['start_time_sec'] >=66600) | (Orange_Closed['start_time_sec'] < 10800)] 

In [11]:
#Red

#Early AM Hours
Red_Early_AM_Open = Red_Open.loc[(Red_Open['start_time_sec'] >= 10800) & (Red_Open['start_time_sec'] < 25200)]
Red_Early_AM_Closed = Red_Closed.loc[(Red_Closed['start_time_sec'] >= 10800) & (Red_Closed['start_time_sec'] < 25200)]

#AM Peak Hours
Red_AM_Peak_Open = Red_Open.loc[(Red_Open['start_time_sec'] >= 25200) & (Red_Open['start_time_sec'] < 32400)]
Red_AM_Peak_Closed = Red_Closed.loc[(Red_Closed['start_time_sec'] >= 25200) & (Red_Closed['start_time_sec'] < 32400)]

#Midday Hours
Red_Midday_Open = Red_Open.loc[(Red_Open['start_time_sec'] >= 32400) & (Red_Open['start_time_sec'] < 57600)]
Red_Midday_Closed = Red_Closed.loc[(Red_Closed['start_time_sec'] >= 32400) & (Red_Closed['start_time_sec'] < 57600)]

#PM Peak Hours
Red_PM_Peak_Open = Red_Open.loc[(Red_Open['start_time_sec'] >= 57600) & (Red_Open['start_time_sec'] < 66600)]
Red_PM_Peak_Closed = Red_Closed.loc[(Red_Closed['start_time_sec'] >= 57600) & (Red_Closed['start_time_sec'] < 66600)]

#Evening Hours
Red_Evening_Open = Red_Open.loc[(Red_Open['start_time_sec'] >= 66600) | (Red_Open['start_time_sec'] < 10800)]
Red_Evening_Closed = Red_Closed.loc[(Red_Closed['start_time_sec'] >=66600) | (Red_Closed['start_time_sec'] < 10800)] 

In [12]:
#Green_B

#Early AM Hours
Green_B_Early_AM_Open = Green_B_Open.loc[(Green_B_Open['start_time_sec'] >= 10800) & (Green_B_Open['start_time_sec'] < 25200)]
Green_B_Early_AM_Closed = Green_B_Closed.loc[(Green_B_Closed['start_time_sec'] >= 10800) & (Green_B_Closed['start_time_sec'] < 25200)]

#AM Peak Hours
Green_B_AM_Peak_Open = Green_B_Open.loc[(Green_B_Open['start_time_sec'] >= 25200) & (Green_B_Open['start_time_sec'] < 32400)]
Green_B_AM_Peak_Closed = Green_B_Closed.loc[(Green_B_Closed['start_time_sec'] >= 25200) & (Green_B_Closed['start_time_sec'] < 32400)]

#Midday Hours
Green_B_Midday_Open = Green_B_Open.loc[(Green_B_Open['start_time_sec'] >= 32400) & (Green_B_Open['start_time_sec'] < 57600)]
Green_B_Midday_Closed = Green_B_Closed.loc[(Green_B_Closed['start_time_sec'] >= 32400) & (Green_B_Closed['start_time_sec'] < 57600)]

#PM Peak Hours
Green_B_PM_Peak_Open = Green_B_Open.loc[(Green_B_Open['start_time_sec'] >= 57600) & (Green_B_Open['start_time_sec'] < 66600)]
Green_B_PM_Peak_Closed = Green_B_Closed.loc[(Green_B_Closed['start_time_sec'] >= 57600) & (Green_B_Closed['start_time_sec'] < 66600)]

#Evening Hours
Green_B_Evening_Open = Green_B_Open.loc[(Green_B_Open['start_time_sec'] >= 66600) | (Green_B_Open['start_time_sec'] < 10800)]
Green_B_Evening_Closed = Green_B_Closed.loc[(Green_B_Closed['start_time_sec'] >=66600) | (Green_B_Closed['start_time_sec'] < 10800)] 

In [13]:
#Green_C

#Early AM Hours
Green_C_Early_AM_Open = Green_C_Open.loc[(Green_C_Open['start_time_sec'] >= 10800) & (Green_C_Open['start_time_sec'] < 25200)]
Green_C_Early_AM_Closed = Green_C_Closed.loc[(Green_C_Closed['start_time_sec'] >= 10800) & (Green_C_Closed['start_time_sec'] < 25200)]

#AM Peak Hours
Green_C_AM_Peak_Open = Green_C_Open.loc[(Green_C_Open['start_time_sec'] >= 25200) & (Green_C_Open['start_time_sec'] < 32400)]
Green_C_AM_Peak_Closed = Green_C_Closed.loc[(Green_C_Closed['start_time_sec'] >= 25200) & (Green_C_Closed['start_time_sec'] < 32400)]

#Midday Hours
Green_C_Midday_Open = Green_C_Open.loc[(Green_C_Open['start_time_sec'] >= 32400) & (Green_C_Open['start_time_sec'] < 57600)]
Green_C_Midday_Closed = Green_C_Closed.loc[(Green_C_Closed['start_time_sec'] >= 32400) & (Green_C_Closed['start_time_sec'] < 57600)]

#PM Peak Hours
Green_C_PM_Peak_Open = Green_C_Open.loc[(Green_C_Open['start_time_sec'] >= 57600) & (Green_C_Open['start_time_sec'] < 66600)]
Green_C_PM_Peak_Closed = Green_C_Closed.loc[(Green_C_Closed['start_time_sec'] >= 57600) & (Green_C_Closed['start_time_sec'] < 66600)]

#Evening Hours
Green_C_Evening_Open = Green_C_Open.loc[(Green_C_Open['start_time_sec'] >= 66600) | (Green_C_Open['start_time_sec'] < 10800)]
Green_C_Evening_Closed = Green_C_Closed.loc[(Green_C_Closed['start_time_sec'] >=66600) | (Green_C_Closed['start_time_sec'] < 10800)] 

In [14]:
#Green_D

#Early AM Hours
Green_D_Early_AM_Open = Green_D_Open.loc[(Green_D_Open['start_time_sec'] >= 10800) & (Green_D_Open['start_time_sec'] < 25200)]
Green_D_Early_AM_Closed = Green_D_Closed.loc[(Green_D_Closed['start_time_sec'] >= 10800) & (Green_D_Closed['start_time_sec'] < 25200)]

#AM Peak Hours
Green_D_AM_Peak_Open = Green_D_Open.loc[(Green_D_Open['start_time_sec'] >= 25200) & (Green_D_Open['start_time_sec'] < 32400)]
Green_D_AM_Peak_Closed = Green_D_Closed.loc[(Green_D_Closed['start_time_sec'] >= 25200) & (Green_D_Closed['start_time_sec'] < 32400)]

#Midday Hours
Green_D_Midday_Open = Green_D_Open.loc[(Green_D_Open['start_time_sec'] >= 32400) & (Green_D_Open['start_time_sec'] < 57600)]
Green_D_Midday_Closed = Green_D_Closed.loc[(Green_D_Closed['start_time_sec'] >= 32400) & (Green_D_Closed['start_time_sec'] < 57600)]

#PM Peak Hours
Green_D_PM_Peak_Open = Green_D_Open.loc[(Green_D_Open['start_time_sec'] >= 57600) & (Green_D_Open['start_time_sec'] < 66600)]
Green_D_PM_Peak_Closed = Green_D_Closed.loc[(Green_D_Closed['start_time_sec'] >= 57600) & (Green_D_Closed['start_time_sec'] < 66600)]

#Evening Hours
Green_D_Evening_Open = Green_D_Open.loc[(Green_D_Open['start_time_sec'] >= 66600) | (Green_D_Open['start_time_sec'] < 10800)]
Green_D_Evening_Closed = Green_D_Closed.loc[(Green_D_Closed['start_time_sec'] >=66600) | (Green_D_Closed['start_time_sec'] < 10800)] 

In [15]:
#Green_E

#Early AM Hours
Green_E_Early_AM_Open = Green_E_Open.loc[(Green_E_Open['start_time_sec'] >= 10800) & (Green_E_Open['start_time_sec'] < 25200)]
Green_E_Early_AM_Closed = Green_E_Closed.loc[(Green_E_Closed['start_time_sec'] >= 10800) & (Green_E_Closed['start_time_sec'] < 25200)]

#AM Peak Hours
Green_E_AM_Peak_Open = Green_E_Open.loc[(Green_E_Open['start_time_sec'] >= 25200) & (Green_E_Open['start_time_sec'] < 32400)]
Green_E_AM_Peak_Closed = Green_E_Closed.loc[(Green_E_Closed['start_time_sec'] >= 25200) & (Green_E_Closed['start_time_sec'] < 32400)]

#Midday Hours
Green_E_Midday_Open = Green_E_Open.loc[(Green_E_Open['start_time_sec'] >= 32400) & (Green_E_Open['start_time_sec'] < 57600)]
Green_E_Midday_Closed = Green_E_Closed.loc[(Green_E_Closed['start_time_sec'] >= 32400) & (Green_E_Closed['start_time_sec'] < 57600)]

#PM Peak Hours
Green_E_PM_Peak_Open = Green_E_Open.loc[(Green_E_Open['start_time_sec'] >= 57600) & (Green_E_Open['start_time_sec'] < 66600)]
Green_E_PM_Peak_Closed = Green_E_Closed.loc[(Green_E_Closed['start_time_sec'] >= 57600) & (Green_E_Closed['start_time_sec'] < 66600)]

#Evening Hours
Green_E_Evening_Open = Green_E_Open.loc[(Green_E_Open['start_time_sec'] >= 66600) | (Green_E_Open['start_time_sec'] < 10800)]
Green_E_Evening_Closed = Green_E_Closed.loc[(Green_E_Closed['start_time_sec'] >=66600) | (Green_E_Closed['start_time_sec'] < 10800)] 

Data for model

In [16]:
f = ['Heath Street','Back of the Hill','Riverway','Mission Park','Fenwood Road',
     'Brigham Circle', 'Longwood Medical Area','Museum of Fine Arts',
     'Northeastern University','Symphony','Prudential','Copley','Arlington',
     'Boylston','Park Street','Government Center','Haymarket','North Station',
     'Science Park/West End']

t = ['Back of the Hill','Riverway','Mission Park','Fenwood Road',
     'Brigham Circle', 'Longwood Medical Area','Museum of Fine Arts',
     'Northeastern University','Symphony','Prudential','Copley','Arlington',
     'Boylston','Park Street','Government Center','Haymarket','North Station',
     'Science Park/West End','Lechmere']

Green_E = pd.DataFrame(f, columns = ['from'])
Green_E['to'] = t

In [17]:
f = ['Riverside', 'Woodland', 'Waban', 'Eliot',
       'Newton Highlands', 'Newton Centre', 'Chestnut Hill', 'Reservoir',
       'Beaconsfield', 'Brookline Hills', 'Brookline Village', 'Longwood',
       'Fenway','Kenmore', 'Hynes Convention Center', 'Copley', 'Arlington',
       'Boylston','Park Street', 'Government Center', 'Haymarket',
       'North Station', 'Science Park/West End']

t = ['Woodland', 'Waban', 'Eliot',
       'Newton Highlands', 'Newton Centre', 'Chestnut Hill', 'Reservoir',
       'Beaconsfield', 'Brookline Hills', 'Brookline Village', 'Longwood',
       'Fenway','Kenmore', 'Hynes Convention Center', 'Copley', 'Arlington',
       'Boylston','Park Street', 'Government Center', 'Haymarket',
       'North Station', 'Science Park/West End', 'Lechmere']

Green_D = pd.DataFrame(f, columns = ['from'])
Green_D['to'] = t

In [18]:
f = ['Boston College', 'South Street',
       'Chestnut Hill Avenue', 'Chiswick Road', 'Sutherland Road',
       'Washington Street', 'Warren Street', 'Allston Street',
       'Griggs Street', 'Harvard Avenue', "Packard's Corner",'Babcock Street', 'Amory Street','Boston University Central', 'Boston University East',
       'Blandford Street','Kenmore','Hynes Convention Center', 'Copley',
       'Arlington', 'Boylston', 'Park Street']

t = ['South Street','Chestnut Hill Avenue', 'Chiswick Road', 'Sutherland Road',
       'Washington Street', 'Warren Street', 'Allston Street',
       'Griggs Street', 'Harvard Avenue', "Packard's Corner",'Babcock Street', 'Amory Street','Boston University Central', 'Boston University East',
       'Blandford Street','Kenmore','Hynes Convention Center', 'Copley',
       'Arlington', 'Boylston', 'Park Street', 'Government Center']

Green_B = pd.DataFrame(f, columns = ['from'])
Green_B ['to'] = t

In [19]:
f = ['Cleveland Circle','Englewood Avenue','Dean Road','Tappan Street','Washington Square',
     'Fairbanks Street', 'Brandon Hall','Summit Avenue','Coolidge Corner','Saint Paul Street',
     'Kent Street','Hawes Street',"Saint Mary's Street",'Kenmore', 'Hynes Convention Center', 'Copley', 'Arlington',
       'Boylston', 'Park Street', 'Government Center', 'Haymarket',
       'North Station']

t = ['Englewood Avenue','Dean Road','Tappan Street','Washington Square',
     'Fairbanks Street', 'Brandon Hall','Summit Avenue','Coolidge Corner','Saint Paul Street',
     'Kent Street','Hawes Street',"Saint Mary's Street",'Kenmore', 'Hynes Convention Center', 'Copley', 'Arlington',
       'Boylston', 'Park Street', 'Government Center', 'Haymarket',
       'North Station', 'Science Park/West End']

Green_C = pd.DataFrame(f, columns = ['from'])
Green_C ['to'] = t

In [20]:
f = ['Forest Hills', 'Green Street', 'Stony Brook', 'Jackson Square',
       'Roxbury Crossing', 'Ruggles', 'Massachusetts Avenue', 'Back Bay',
       'Tufts Medical Center', 'Chinatown', 'Downtown Crossing', 'State',
       'Haymarket', 'North Station', 'Community College',
       'Sullivan Square', 'Assembly','Wellington', 'Malden Center', 
       ]

t = [ 'Green Street', 'Stony Brook', 'Jackson Square',
       'Roxbury Crossing', 'Ruggles', 'Massachusetts Avenue', 'Back Bay',
       'Tufts Medical Center', 'Chinatown', 'Downtown Crossing', 'State',
       'Haymarket', 'North Station', 'Community College',
       'Sullivan Square', 'Assembly','Wellington', 'Malden Center', 'Oak Grove',
       ]
Orange = pd.DataFrame(f, columns = ['from'])
Orange ['to'] = t

In [21]:
t = ['Alewife', 'Davis', 'Porter', 'Harvard', 'Central', 'Kendall/MIT',
       'Charles/MGH', 'Park Street', 'Downtown Crossing','South Station',
       'Broadway', 'Andrew']

f = ['Davis', 'Porter', 'Harvard', 'Central', 'Kendall/MIT',
       'Charles/MGH', 'Park Street', 'Downtown Crossing','South Station',
       'Broadway', 'Andrew', 'JFK/UMass']

Red = pd.DataFrame(f, columns = ['from'])
Red ['to'] = t

In [22]:
Orange['mean'] = 0
Green_B['mean'] = 0
Green_C['mean'] = 0
Green_D['mean'] = 0
Green_E['mean'] = 0
Red['mean'] = 0

Orange['std'] = 0
Green_B['std'] = 0
Green_C['std'] = 0
Green_D['std'] = 0
Green_E['std'] = 0
Red['std'] = 0

Orange['mean_closed'] = 0
Green_B['mean_closed'] = 0
Green_C['mean_closed'] = 0
Green_D['mean_closed'] = 0
Green_E['mean_closed'] = 0
Red['mean_closed'] = 0

Orange['std_closed'] = 0
Green_B['std_closed'] = 0
Green_C['std_closed'] = 0
Green_D['std_closed'] = 0
Green_E['std_closed'] = 0
Red['std_closed'] = 0

In [23]:
o1 = Orange
o2 = Orange
o3 = Orange
o4 = Orange
o5 = Orange

gb1 = Green_B
gb2 = Green_B
gb3 = Green_B
gb4 = Green_B
gb5 = Green_B

gc1 = Green_C
gc2 = Green_C
gc3 = Green_C
gc4 = Green_C
gc5 = Green_C

gd1 = Green_D
gd2 = Green_D
gd3 = Green_D
gd4 = Green_D
gd5 = Green_D

ge1 = Green_E
ge2 = Green_E
ge3 = Green_E
ge4 = Green_E
ge5 = Green_E

r1 = Red
r2 = Red
r3 = Red
r4 = Red
r5 = Red

In [24]:
def fill_mean(new, open, col):
  for start, stop, i in zip(new['from'], new['to'], range(len(new))):
    new.iloc[i,col] = open.loc[(open['from_stop_name']==start) & (open['to_stop_name']==stop)]['travel_time_sec'].mean()
  return new

In [25]:
def fill_std(new, open, col):
  for start, stop, i in zip(new['from'], new['to'], range(len(new))):
    new.iloc[i,col] = open.loc[(open['from_stop_name']==start) & (open['to_stop_name']==stop)]['travel_time_sec'].std()
  return new

In [26]:
Green_B = fill_mean(Green_B, Green_B_Open,2)
Green_C = fill_mean(Green_C, Green_C_Open,2)
Green_D = fill_mean(Green_D, Green_D_Open,2)
Green_E = fill_mean(Green_E, Green_E_Open,2)
Red = fill_mean(Red, Red_Open, 2)
Orange = fill_mean(Orange, Orange_Open, 2)

In [27]:
Green_B = fill_std(Green_B, Green_B_Open,3)
Green_C = fill_std(Green_C, Green_C_Open,3)
Green_D = fill_std(Green_D, Green_D_Open,3)
Green_E = fill_std(Green_E, Green_E_Open,3)
Red = fill_std(Red, Red_Open, 3)
Orange = fill_std(Orange, Orange_Open, 3)

In [28]:
Green_B = fill_mean(Green_B, Green_B_Open,4)
Green_C = fill_mean(Green_C, Green_C_Open,4)
Green_D = fill_mean(Green_D, Green_D_Open,4)
Green_E = fill_mean(Green_E, Green_E_Open,4)
Red = fill_mean(Red, Red_Open, 4)
Orange = fill_mean(Orange, Orange_Open, 4)

In [29]:
Green_B = fill_std(Green_B, Green_B_Open,5)
Green_C = fill_std(Green_C, Green_C_Open,5)
Green_D = fill_std(Green_D, Green_D_Open,5)
Green_E = fill_std(Green_E, Green_E_Open,5)
Red = fill_std(Red, Red_Open, 5)
Orange = fill_std(Orange, Orange_Open, 5)

In [30]:
Red.iloc[7,2] =120
Red.iloc[7,4] = 120
Red.iloc[7,3] = Red['std'].mean()
Red.iloc[7,5] = Red['std_closed'].mean()

In [31]:
Green_B.iloc[0,4] = Green_B.iloc[0,2] 
Green_B.iloc[0,5] = Green_B.iloc[0,3] 
Green_C.iloc[[0,19,20,21],4] = Green_C.iloc[[0,19,20,21],2] 
Green_C.iloc[[0,19,20,21],5] = Green_C.iloc[[0,19,20,21],3] 
Green_D.iloc[22,4] = Green_D.iloc[22,2] 
Green_D.iloc[22,5] = Green_D.iloc[22,3] 
Green_E.iloc[18,4] = Green_E.iloc[18,2] 
Green_E.iloc[18,5] = Green_E.iloc[18,3] 

In [37]:
Orange.to_csv('Orange_Format.csv')
Red.to_csv('Red_Format.csv')
Green_C.to_csv('Green_C_Format.csv')
Green_B.to_csv('Green_B_Format.csv')
Green_D.to_csv('Green_D_Format.csv')
Green_E.to_csv('Green_E_Format.csv')

In [32]:
o = [o1,o2,o3,o4,o5]
times = [Orange_Early_AM_Open, Orange_AM_Peak_Open, Orange_Midday_Open, Orange_PM_Peak_Open, Orange_Evening_Open]
times_c = [Orange_Early_AM_Closed, Orange_AM_Peak_Closed, Orange_Midday_Closed, Orange_PM_Peak_Closed, Orange_Evening_Closed]

for train,time in zip(o,times):
  train = fill_mean(train, time, 2)
  train = fill_std(train, time, 3)

for train,time in zip(o,times_c):
  train = fill_mean(train, time, 4)
  train = fill_std(train, time, 5)

In [33]:
r = [r1,r2,r3,r4,r5]
times = [Red_Early_AM_Open, Red_AM_Peak_Open, Red_Midday_Open, Red_PM_Peak_Open, Red_Evening_Open]
times_c = [Red_Early_AM_Closed, Red_AM_Peak_Closed, Red_Midday_Closed, Red_PM_Peak_Closed, Red_Evening_Closed]

for train,time in zip(r,times):
  train = fill_mean(train, time, 2)
  train = fill_std(train, time, 3)

for train,time in zip(r,times_c):
  train = fill_mean(train, time, 4)
  train = fill_std(train, time, 5)

In [35]:
r2.iloc[7,2] = 120
r2.iloc[7,4] = 120
r2.iloc[7,3] = r1['std'].mean()
r2.iloc[7,4] = r1['std_closed'].mean()

r4.iloc[7,2] = 120
r4.iloc[7,4] = 120
r4.iloc[7,3] = r1['std'].mean()
r4.iloc[7,4] = r1['std_closed'].mean()

In [36]:
gb = [gb1,gb2,gb3,gb4,gb5]
times = [Green_B_Early_AM_Open, Green_B_AM_Peak_Open, Green_B_Midday_Open, Green_B_PM_Peak_Open, Green_B_Evening_Open]
times_c = [Green_B_Early_AM_Closed, Green_B_AM_Peak_Closed, Green_B_Midday_Closed, Green_B_PM_Peak_Closed, Green_B_Evening_Closed]

for train,time in zip(gb,times):
  train = fill_mean(train, time, 2)
  train = fill_std(train, time, 3)

for train,time in zip(gb,times_c):
  train = fill_mean(train, time, 4)
  train = fill_std(train, time, 5)

In [39]:
gb2.iloc[0,4] = gb2.iloc[0,2]
gb2.iloc[0,5] = gb2.iloc[0,3]

gb4.iloc[0,4] = gb4.iloc[0,2]
gb4.iloc[0,5] = gb4.iloc[0,3]

In [40]:
gc = [gc1,gc2,gc3,gc4,gc5]
times = [Green_C_Early_AM_Open, Green_C_AM_Peak_Open, Green_C_Midday_Open, Green_C_PM_Peak_Open, Green_C_Evening_Open]
times_c = [Green_C_Early_AM_Closed, Green_C_AM_Peak_Closed, Green_C_Midday_Closed, Green_C_PM_Peak_Closed, Green_C_Evening_Closed]

for train,time in zip(gc,times):
  train = fill_mean(train, time, 2)
  train = fill_std(train, time, 3)

for train,time in zip(gc,times_c):
  train = fill_mean(train, time, 4)
  train = fill_std(train, time, 5)

In [ ]:
gc2.iloc[[0,19,20,21],4] = gc2.iloc[[0,19,20,21],2]
gc2.iloc[[0,19,20,21],5] = gc2.iloc[[0,19,20,21],3]

gc4.iloc[[0,19,20,21],4] = gc4.iloc[[0,19,20,21],2]
gc4.iloc[[0,19,20,21],5] = gc4.iloc[[0,19,20,21],3]

In [43]:
gd = [gd1,gd2,gd3,gd4,gd5]
times = [Green_D_Early_AM_Open, Green_D_AM_Peak_Open, Green_D_Midday_Open, Green_D_PM_Peak_Open, Green_D_Evening_Open]
times_c = [Green_D_Early_AM_Closed, Green_D_AM_Peak_Closed, Green_D_Midday_Closed, Green_D_PM_Peak_Closed, Green_D_Evening_Closed]

for train,time in zip(gd,times):
  train = fill_mean(train, time, 2)
  train = fill_std(train, time, 3)

for train,time in zip(gd,times_c):
  train = fill_mean(train, time, 4)
  train = fill_std(train, time, 5)

In [45]:
ge = [ge1,ge2,ge3,ge4,ge5]
times = [Green_E_Early_AM_Open, Green_E_AM_Peak_Open, Green_E_Midday_Open, Green_E_PM_Peak_Open, Green_E_Evening_Open]
times_c = [Green_E_Early_AM_Closed, Green_E_AM_Peak_Closed, Green_E_Midday_Closed, Green_E_PM_Peak_Closed, Green_E_Evening_Closed]

for train,time in zip(ge,times):
  train = fill_mean(train, time, 2)
  train = fill_std(train, time, 3)

for train,time in zip(ge,times_c):
  train = fill_mean(train, time, 4)
  train = fill_std(train, time, 5)

Busses

In [110]:
bus_7 = pd.read_csv('MBTA-Bus-Arrival-Departure-Times_2022-07.csv')
bus_8 = pd.read_csv('MBTA-Bus-Arrival-Departure-Times_2022-08.csv')
bus_9 = pd.read_csv('MBTA-Bus-Arrival-Departure-Times_2022-09.csv')

In [129]:
Bus_open = bus_7.append(bus_8.loc[bus_8['service_date']<='2022-08-19'])
Bus_open = Bus_open.append(bus_9.loc[bus_9['service_date']> '2022-09-18'])

In [130]:
Bus_closed = bus_8.loc[bus_8['service_date']>'2022-08-19']
Bus_closed = Bus_closed.append(bus_9.loc[bus_9['service_date']<='2022-09-18'])

In [131]:
Bus_open = Bus_open[Bus_open['route_id'].isin(['137','104','92','86','501'])]
Bus_closed = Bus_closed[Bus_closed['route_id'].isin(['137','104','92','86','501'])]

In [132]:
Bus_open = Bus_open.merge(Stops[['stop_code','stop_name']], how = 'left', left_on = 'stop_id', right_on = 'stop_code')
Bus_open.drop(columns = ['stop_code'], inplace = True)
Bus_open = Bus_open.rename({'stop_name': 'from_stop_name'}, axis = 1)

In [133]:
Bus_closed = Bus_closed.merge(Stops[['stop_code','stop_name']], how = 'left', left_on = 'stop_id', right_on = 'stop_code')
Bus_closed.drop(columns = ['stop_code'], inplace = True)
Bus_closed = Bus_closed.rename({'stop_name': 'from_stop_name'}, axis = 1)

In [134]:
Bus_open.drop(columns = ['half_trip_id','time_point_id','scheduled_headway', 'headway'], inplace = True)
Bus_closed.drop(columns = ['half_trip_id','time_point_id','scheduled_headway', 'headway'], inplace = True)

In [135]:
Bus_open['scheduled']=pd.to_datetime(Bus_open['scheduled'].astype(str))
Bus_open['actual']=pd.to_datetime(Bus_open['actual'].astype(str))

Bus_closed['scheduled']=pd.to_datetime(Bus_closed['scheduled'].astype(str))
Bus_closed['actual']=pd.to_datetime(Bus_closed['actual'].astype(str))

In [136]:
B_137 = Bus_open.loc[Bus_open['route_id']=='137']
B_104 = Bus_open.loc[Bus_open['route_id']=='104']
B_92 = Bus_open.loc[Bus_open['route_id']=='92']
B_86 = Bus_open.loc[Bus_open['route_id']=='86']
B_501 = Bus_open.loc[Bus_open['route_id']=='501']

B_137_c = Bus_closed.loc[Bus_closed['route_id']=='137']
B_104_c = Bus_closed.loc[Bus_closed['route_id']=='104']
B_92_c = Bus_closed.loc[Bus_closed['route_id']=='92']
B_86_c = Bus_closed.loc[Bus_closed['route_id']=='86']
B_501_c = Bus_closed.loc[Bus_closed['route_id']=='501']


In [ ]:
#change scheduled and actual to just times
from datetime import *  
from datetime import datetime  
import datetime as dt  
import time

#B_137
B_137['scheduled'] = pd.to_datetime(B_137['scheduled'])
B_137['scheduled'] = B_137['scheduled'].dt.strftime('%H:%M:%S')
B_137['actual'] = pd.to_datetime(B_137['actual'])
B_137['actual'] = B_137['actual'].dt.strftime('%H:%M:%S')
B_137_c['scheduled'] = pd.to_datetime(B_137_c['scheduled'])
B_137_c['scheduled'] = B_137_c['scheduled'].dt.strftime('%H:%M:%S')
B_137_c['actual'] = pd.to_datetime(B_137_c['actual'])
B_137_c['actual'] = B_137_c['actual'].dt.strftime('%H:%M:%S')

#B_104
B_104['scheduled'] = pd.to_datetime(B_104['scheduled'])
B_104['scheduled'] = B_104['scheduled'].dt.strftime('%H:%M:%S')
B_104['actual'] = pd.to_datetime(B_137['actual'])
B_104['actual'] = B_104['actual'].dt.strftime('%H:%M:%S')
B_104_c['scheduled'] = pd.to_datetime(B_104_c['scheduled'])
B_104_c['scheduled'] = B_104_c['scheduled'].dt.strftime('%H:%M:%S')
B_104_c['actual'] = pd.to_datetime(B_104_c['actual'])
B_104_c['actual'] = B_104_c['actual'].dt.strftime('%H:%M:%S')

#B_92
B_92['scheduled'] = pd.to_datetime(B_92['scheduled'])
B_92['scheduled'] = B_92['scheduled'].dt.strftime('%H:%M:%S')
B_92['actual'] = pd.to_datetime(B_92['actual'])
B_92['actual'] = B_92['actual'].dt.strftime('%H:%M:%S')
B_92_c['scheduled'] = pd.to_datetime(B_92_c['scheduled'])
B_92_c['scheduled'] = B_92_c['scheduled'].dt.strftime('%H:%M:%S')
B_92_c['actual'] = pd.to_datetime(B_92_c['actual'])
B_92_c['actual'] = B_92_c['actual'].dt.strftime('%H:%M:%S')

#B_86
B_86['scheduled'] = pd.to_datetime(B_86['scheduled'])
B_86['scheduled'] = B_86['scheduled'].dt.strftime('%H:%M:%S')
B_86['actual'] = pd.to_datetime(B_86['actual'])
B_86['actual'] = B_86['actual'].dt.strftime('%H:%M:%S')
B_86_c['scheduled'] = pd.to_datetime(B_86_c['scheduled'])
B_86_c['scheduled'] = B_86_c['scheduled'].dt.strftime('%H:%M:%S')
B_86_c['actual'] = pd.to_datetime(B_86_c['actual'])
B_86_c['actual'] = B_86_c['actual'].dt.strftime('%H:%M:%S')


In [ ]:
#B_92
B_501['scheduled'] = pd.to_datetime(B_501['scheduled'])
B_501['scheduled'] = B_501['scheduled'].dt.strftime('%H:%M:%S')
B_501['actual'] = pd.to_datetime(B_501['actual'])
B_501['actual'] = B_501['actual'].dt.strftime('%H:%M:%S')
B_501_c['scheduled'] = pd.to_datetime(B_501_c['scheduled'])
B_501_c['scheduled'] = B_501_c['scheduled'].dt.strftime('%H:%M:%S')
B_501_c['actual'] = pd.to_datetime(B_501_c['actual'])
B_501_c['actual'] = B_501_c['actual'].dt.strftime('%H:%M:%S')

In [ ]:
#B_137
B_137['scheduled'] = pd.to_timedelta(B_137['scheduled'])
B_137['scheduled_seconds'] = B_137['scheduled'].dt.total_seconds()
B_137['actual'] = pd.to_timedelta(B_137['actual'])
B_137['actual_seconds'] = B_137['actual'].dt.total_seconds()
B_137[['actual_seconds', 'scheduled_seconds']].astype(float)
B_137['deviation'] = B_137['actual_seconds'] - B_137['scheduled_seconds']

B_137_c['scheduled'] = pd.to_timedelta(B_137_c['scheduled'])
B_137_c['scheduled_seconds'] = B_137_c['scheduled'].dt.total_seconds()
B_137_c['actual'] = pd.to_timedelta(B_137_c['actual'])
B_137_c['actual_seconds'] = B_137_c['actual'].dt.total_seconds()
B_137_c[['actual_seconds', 'scheduled_seconds']].astype(float)
B_137_c['deviation'] = B_137_c['actual_seconds'] - B_137_c['scheduled_seconds']

#B_104
B_104['scheduled'] = pd.to_timedelta(B_104['scheduled'])
B_104['scheduled_seconds'] = B_104['scheduled'].dt.total_seconds()
B_104['actual'] = pd.to_timedelta(B_104['actual'])
B_104['actual_seconds'] = B_104['actual'].dt.total_seconds()
B_104[['actual_seconds', 'scheduled_seconds']].astype(float)
B_104['deviation'] = B_104['actual_seconds'] - B_104['scheduled_seconds']

B_104_c['scheduled'] = pd.to_timedelta(B_104_c['scheduled'])
B_104_c['scheduled_seconds'] = B_104_c['scheduled'].dt.total_seconds()
B_104_c['actual'] = pd.to_timedelta(B_104_c['actual'])
B_104_c['actual_seconds'] = B_104_c['actual'].dt.total_seconds()
B_104_c[['actual_seconds', 'scheduled_seconds']].astype(float)
B_104_c['deviation'] = B_104_c['actual_seconds'] - B_104_c['scheduled_seconds']

#B_92
B_92['scheduled'] = pd.to_timedelta(B_92['scheduled'])
B_92['scheduled_seconds'] = B_92['scheduled'].dt.total_seconds()
B_92['actual'] = pd.to_timedelta(B_92['actual'])
B_92['actual_seconds'] = B_92['actual'].dt.total_seconds()
B_92[['actual_seconds', 'scheduled_seconds']].astype(float)
B_92['deviation'] = B_92['actual_seconds'] - B_92['scheduled_seconds']

B_92_c['scheduled'] = pd.to_timedelta(B_92_c['scheduled'])
B_92_c['scheduled_seconds'] = B_92_c['scheduled'].dt.total_seconds()
B_92_c['actual'] = pd.to_timedelta(B_92_c['actual'])
B_92_c['actual_seconds'] = B_92_c['actual'].dt.total_seconds()
B_92_c[['actual_seconds', 'scheduled_seconds']].astype(float)
B_92_c['deviation'] = B_92_c['actual_seconds'] - B_92_c['scheduled_seconds']

#B_86
B_86['scheduled'] = pd.to_timedelta(B_86['scheduled'])
B_86['scheduled_seconds'] = B_86['scheduled'].dt.total_seconds()
B_86['actual'] = pd.to_timedelta(B_86['actual'])
B_86['actual_seconds'] = B_86['actual'].dt.total_seconds()
B_86[['actual_seconds', 'scheduled_seconds']].astype(float)
B_86['deviation'] = B_86['actual_seconds'] - B_86['scheduled_seconds']

B_86_c['scheduled'] = pd.to_timedelta(B_86_c['scheduled'])
B_86_c['scheduled_seconds'] = B_86_c['scheduled'].dt.total_seconds()
B_86_c['actual'] = pd.to_timedelta(B_86_c['actual'])
B_86_c['actual_seconds'] = B_86_c['actual'].dt.total_seconds()
B_86_c[['actual_seconds', 'scheduled_seconds']].astype(float)
B_86_c['deviation'] = B_86_c['actual_seconds'] - B_86_c['scheduled_seconds']

#B_501
B_501['scheduled'] = pd.to_timedelta(B_501['scheduled'])
B_501['scheduled_seconds'] = B_501['scheduled'].dt.total_seconds()
B_501['actual'] = pd.to_timedelta(B_501['actual'])
B_501['actual_seconds'] = B_501['actual'].dt.total_seconds()
B_501[['actual_seconds', 'scheduled_seconds']].astype(float)
B_501['deviation'] = B_501['actual_seconds'] - B_501['scheduled_seconds']

B_501_c['scheduled'] = pd.to_timedelta(B_501_c['scheduled'])
B_501_c['scheduled_seconds'] = B_501_c['scheduled'].dt.total_seconds()
B_501_c['actual'] = pd.to_timedelta(B_501_c['actual'])
B_501_c['actual_seconds'] = B_501_c['actual'].dt.total_seconds()
B_501_c[['actual_seconds', 'scheduled_seconds']].astype(float)
B_501_c['deviation'] = B_501_c['actual_seconds'] - B_501_c['scheduled_seconds']


In [144]:
B_137.to_csv('B_137_open.csv')
B_104.to_csv('B_104_open.csv')
B_92.to_csv('B_92_open.csv')
B_86.to_csv('B_86_open.csv')
B_501.to_csv('B_501_open.csv')

B_137_c.to_csv('B_137_closed.csv')
B_104_c.to_csv('B_104_closed.csv')
B_92_c.to_csv('B_92_closed.csv')
B_86_c.to_csv('B_86_closed.csv')
B_501_c.to_csv('B_501_closed.csv')

Model

In [73]:
def model(df, name, model_dict):
  state_space = StateSpace.from_range(0, 240)
  identity = matrix.identity(state_space)
  
  n = Network(state_space)
  edge_stats = df

  nodes = set(edge_stats['from'].tolist()).union(set(edge_stats['to'].tolist()))
  
  for node in nodes:
    n.add_node(node, identity)
  
  for idx, edge in edge_stats.iterrows():
    mx = matrix.truncnorm(state_space, mean=0, std=edge['std'])
    n.add_edge(edge['from'], edge['to'], mx)
  
  model_dict[name] = n
  return model_dict

In [107]:
model_dict = {}
model_dict = model(Orange, 'orange', model_dict)
model_dict = model(Green_B, 'green_b', model_dict)
model_dict = model(Green_C, 'green_c', model_dict)
model_dict = model(Green_D, 'green_d', model_dict)
model_dict = model(Green_E, 'green_e', model_dict)
model_dict = model(Red, 'red', model_dict)

Just orange line example

In [46]:
state_space = StateSpace.from_range(-60, 120)
identity = matrix.identity(state_space)

n = Network(state_space)
edge_stats = Orange
nodes = set(edge_stats['from'].tolist()).union(set(edge_stats['to'].tolist()))

for node in nodes:
  n.add_node(node, identity)

for idx, edge in edge_stats.iterrows():
  mx = matrix.truncnorm(state_space, mean=0, std=edge['std'])
  n.add_edge(edge['from'], edge['to'], mx)

In [48]:
stats = []
p0 = vector.unit(state_space, '0')

path = nx.shortest_path(n, source="Forest Hills", target="Oak Grove")

vecs = n.traverse(path, p0)
means = []
stds = []
for v in vecs:
  average = np.average(state_space.values, weights=v)
  # Fast and numerically precise:
  variance = np.average((state_space.values-average)**2, weights=v)
  std = math.sqrt(variance)
  means.append(average)
  stds.append(std)
  df = pd.DataFrame(zip(means, stds), columns=['mean', 'std']).reset_index()
  df.columns = ['element', 'mean', 'std']
  df['train'] = 'orange'
  stats.append(df)

allstats = pd.concat(stats)
allstats

,element,mean,std,train
0,0,0.000000,0.000000,orange
0,0,0.000000,0.000000,orange
1,1,0.000000,0.000000,orange
0,0,0.000000,0.000000,orange
1,1,0.000000,0.000000,orange
...,...,...,...,...
34,34,28.814909,57.662392,orange
35,35,28.814909,57.662392,orange
36,36,29.372550,63.312230,orange
37,37,29.372550,63.312230,orange


In [49]:
alt.Chart(allstats).mark_line().encode(
    alt.X('element:O', title='Step'),
    alt.Y('mean:Q', title='Modelled Mean Schedule Deviation'),
    color=alt.Color('train:N', title='Train')
).properties(
    width = 800,
    height=300
).configure(font='Raleway').configure_view(strokeWidth=0).configure_axis(grid=False, domain=False)

alt.Chart(...)

Orange and Green line

In [51]:
stats = []
p0 = vector.unit(state_space, '0')
trains = ['orange', 'green_d', 'green_e']
for t in trains:
    if(t == 'orange'):
      path = nx.shortest_path(model_dict[t], source="North Station", target="Assembly")
    if(t == 'green_d' or t == 'green_e'):
      path = nx.shortest_path(model_dict[t], source="Copley", target="North Station")
    else:
      continue

    vecs = model_dict[t].traverse(path, p0)
    means = []
    stds = []
    for v in vecs:
        average = np.average(state_space.values, weights=v)
        # Fast and numerically precise:
        variance = np.average((state_space.values-average)**2, weights=v)
        std = math.sqrt(variance)
        means.append(average)
        stds.append(std)
    df = pd.DataFrame(zip(means, stds), columns=['mean', 'std']).reset_index()
    df.columns = ['element', 'mean', 'std']
    df['train'] = t
    stats.append(df)
allstats = pd.concat(stats)
allstats

,element,mean,std,train
0,0,0.000000,0.000000,green_d
1,1,0.000000,0.000000,green_d
2,2,12.561586,9.108081,green_d
3,3,12.561586,9.108081,green_d
4,4,22.728002,26.447038,green_d
5,5,22.728002,26.447038,green_d
6,6,28.977920,26.015319,green_d
7,7,28.977920,26.015319,green_d
8,8,35.951407,33.631155,green_d
9,9,35.951407,33.631155,green_d


In [52]:
alt.Chart(allstats).mark_line().encode(
    alt.X('element:O', title='Step'),
    alt.Y('mean:Q', title='Modelled Mean Schedule Deviation'),
    color=alt.Color('train:N', title='Train')
).properties(
    width = 800,
    height=300
).configure(font='Raleway').configure_view(strokeWidth=0).configure_axis(grid=False, domain=False)

alt.Chart(...)

Renaming Stops

In [125]:
Rnew = Red[0:9]

In [117]:
f_num = ['1','2','3','4','5','6','7','8','9']
t_num = ['2','3','4','5','6','7','8','9', '10']

In [ ]:
Rnew['from'] = f_num

In [ ]:
Rnew['to'] = t_num

In [ ]:
Gdnew = Green_D[0:9]
Gdnew['from'] = f_num
Gdnew['to'] = t_num

Genew = Green_E[0:9]
Genew['from'] = f_num
Genew['to'] = t_num

In [142]:
model_dict = {}
model_dict = model(Orange, 'orange', model_dict)
model_dict = model(Rnew, 'red', model_dict)
model_dict = model(Genew, 'green_e', model_dict)
model_dict = model(Gdnew, 'green_d', model_dict)

In [143]:
stats = []
p0 = vector.unit(state_space, '0')
trains = ['orange', 'green_d', 'green_e', 'red']
for t in trains:
    if(t == 'orange'):
      path = nx.shortest_path(model_dict[t], source="Forest Hills", target="Downtown Crossing")
    else:
      path = nx.shortest_path(model_dict[t], source="1", target="9")

    vecs = model_dict[t].traverse(path, p0)
    means = []
    stds = []
    for v in vecs:
        average = np.average(state_space.values, weights=v)
        # Fast and numerically precise:
        variance = np.average((state_space.values-average)**2, weights=v)
        std = math.sqrt(variance)
        means.append(average)
        stds.append(std)
    df = pd.DataFrame(zip(means, stds), columns=['mean', 'std']).reset_index()
    df.columns = ['element', 'mean', 'std']
    df['train'] = t
    stats.append(df)
allstats = pd.concat(stats)
allstats

,element,mean,std,train
0,0,0.000000,0.000000,orange
1,1,0.000000,0.000000,orange
2,2,18.184889,13.358000,orange
3,3,18.184889,13.358000,orange
4,4,22.299073,21.362941,orange
...,...,...,...,...
12,12,59.412895,42.212467,red
13,13,59.412895,42.212467,red
14,14,59.909075,38.978133,red
15,15,59.909075,38.978133,red


In [144]:
alt.Chart(allstats).mark_line().encode(
    alt.X('element:O', title='Step'),
    alt.Y('mean:Q', title='Modelled Mean Schedule Deviation'),
    color=alt.Color('train:N', title='Train')
).properties(
    width = 800,
    height=300
).configure(font='Raleway').configure_view(strokeWidth=0).configure_axis(grid=False, domain=False)

alt.Chart(...)

In [149]:
Gdnew

,from,to,mean,std,mean_closed,std_closed
0,1,2,67.637652,46.722057,62.134265,11.781633
1,2,3,89.065934,21.233960,88.543002,39.403179
2,3,4,85.945055,33.127821,84.652661,9.093888
3,4,5,81.032907,11.686466,81.567839,18.140413
4,5,6,85.678967,9.824888,85.196067,13.064044
5,6,7,158.871508,18.141318,157.958192,17.975722
6,7,8,123.602612,28.854649,123.667231,30.156698
7,8,9,111.654691,23.061801,117.165400,23.826372
8,9,10,98.433628,12.674627,104.192912,20.181504


In [77]:
def new_model(df, name, model_dict):
  state_space = StateSpace.from_range(0, 240)
  identity = matrix.identity(state_space)
  
  n = Network(state_space)
  edge_stats = df

  nodes = set(edge_stats['from'].tolist()).union(set(edge_stats['to'].tolist()))
  
  for node in nodes:
    n.add_node(node, identity)
  
  for idx, edge in edge_stats.iterrows():
    mx = matrix.truncnorm(state_space, mean=0, std=edge['std_closed'])
    n.add_edge(edge['from'], edge['to'], mx)
  
  model_dict[name] = n
  return model_dict

In [152]:
new_model_dict = {}
new_model_dict = new_model(Orange, 'orange', new_model_dict)
new_model_dict = new_model(Rnew, 'red', new_model_dict)
new_model_dict = new_model(Genew, 'green_e', new_model_dict)
new_model_dict = new_model(Gdnew, 'green_d', new_model_dict)

In [153]:
stats = []
p0 = vector.unit(state_space, '0')
for t in trains:
    if(t == 'orange'):
      path = nx.shortest_path(model_dict[t], source="Forest Hills", target="Downtown Crossing")
    else:
      path = nx.shortest_path(model_dict[t], source="1", target="9")
      
    vecs = new_model_dict[t].traverse(path, p0)
    means = []
    stds = []
    for v in vecs:
        average = np.average(state_space.values, weights=v)
        # Fast and numerically precise:
        variance = np.average((state_space.values-average)**2, weights=v)
        std = math.sqrt(variance)
        means.append(average)
        stds.append(std)
    df = pd.DataFrame(zip(means, stds), columns=['mean', 'std']).reset_index()
    df.columns = ['element', 'mean', 'std']
    df['train'] = t
    stats.append(df)
new_allstats = pd.concat(stats)
new_allstats
all_allstats = pd.merge(allstats, new_allstats, on=['element', 'train'])
all_allstats.columns = ['element', 'Mean', 'Standard Deviation', 'train', 'New Mean', 'New Standard Deviation']
all_allstats

,element,Mean,Standard Deviation,train,New Mean,New Standard Deviation
0,0,0.000000,0.000000,orange,0.000000,0.000000
1,1,0.000000,0.000000,orange,0.000000,0.000000
2,2,18.184889,13.358000,orange,8.106723,5.739696
3,3,18.184889,13.358000,orange,8.106723,5.739696
4,4,22.299073,21.362941,orange,10.894402,10.793330
...,...,...,...,...,...,...
67,12,59.412895,42.212467,red,58.813373,42.936750
68,13,59.412895,42.212467,red,58.813373,42.936750
69,14,59.909075,38.978133,red,59.422383,39.266223
70,15,59.909075,38.978133,red,59.422383,39.266223


In [154]:
plot_means = all_allstats[['element', 'train', 'Mean', 'New Mean']].copy()
plot_means.columns = ['element', 'train', 'Before', 'After']
plot_means = plot_means.melt(id_vars=['element', 'train'])
alt.Chart(plot_means).mark_line().encode(
    alt.X('element:O', title='Step'),
    alt.Y('value:Q', title='Mean (min)'),
    color=alt.Color('train:N', title='Train'),
    strokeDash = alt.StrokeDash('variable:N', title='Scenario', sort='-x')
).properties(
    width = 800,
    height=200,
    title='Modelled Mean Schedule Deviaition Before and After Reliability Improvements'
).configure(font='Raleway').configure_view(strokeWidth=0).configure_axis(grid=False, domain=False)

alt.Chart(...)

Morning Peak Model

In [51]:
f_num = ['1','2','3','4','5','6','7','8','9','10','11']
t_num = ['2','3','4','5','6','7','8','9', '10','11','12']

In [49]:
o2_new = o2[0:11]
r2_new = r2[0:11]
gb2_new = gb2[0:11]
gc2_new = gc2[0:11]
gd2_new = gd2[0:11]
ge2_new = ge2[0:11]

In [ ]:
o2_new['from'] = f_num
r2_new['from'] = f_num
gb2_new['from'] = f_num
gc2_new['from'] = f_num
gd2_new['from'] = f_num
ge2_new['from'] = f_num

o2_new['to'] = t_num
r2_new['to'] = t_num
gb2_new['to'] = t_num
gc2_new['to'] = t_num
gd2_new['to'] = t_num
ge2_new['to'] = t_num

In [74]:
model_dict = {}
model_dict = model(o2_new, 'orange', model_dict)
model_dict = model(r2_new, 'red', model_dict)
model_dict = model(gb2_new, 'green_b', model_dict)
model_dict = model(gc2_new, 'green_c', model_dict)
model_dict = model(gd2_new, 'green_d', model_dict)
model_dict = model(ge2_new, 'green_e', model_dict)


In [75]:
stats = []
state_space = StateSpace.from_range(0, 240)
p0 = vector.unit(state_space, '0')
trains = ['green_b','green_c','orange','red','green_d', 'green_e' ]
for t in trains:
    path = nx.shortest_path(model_dict[t], source="1", target="12")
    
    vecs = model_dict[t].traverse(path, p0)
    means = []
    stds = []
    for v in vecs:
        average = np.average(state_space.values, weights=v)
        # Fast and numerically precise:
        variance = np.average((state_space.values-average)**2, weights=v)
        std = math.sqrt(variance)
        means.append(average)
        stds.append(std)
    df = pd.DataFrame(zip(means, stds), columns=['mean', 'std']).reset_index()
    df.columns = ['element', 'mean', 'std']
    df['train'] = t
    stats.append(df)
allstats = pd.concat(stats)
allstats

,element,mean,std,train
0,0,0.000000,0.000000,green_b
1,1,0.000000,0.000000,green_b
2,2,18.143082,13.326409,green_b
3,3,18.143082,13.326409,green_b
4,4,28.239184,31.408107,green_b
...,...,...,...,...
18,18,55.964640,45.246477,green_e
19,19,55.964640,45.246477,green_e
20,20,57.119004,45.482102,green_e
21,21,57.119004,45.482102,green_e


In [76]:
alt.Chart(allstats).mark_line().encode(
    alt.X('element:O', title='Step'),
    alt.Y('mean:Q', title='Modelled Mean Schedule Deviation'),
    color=alt.Color('train:N', title='Train')
).properties(
    width = 800,
    height=500
).configure(font='Raleway').configure_view(strokeWidth=0).configure_axis(grid=False, domain=False)

alt.Chart(...)

In [81]:
r2_new.iloc[7,5]=0.1

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [82]:
new_model_dict = {}
new_model_dict = new_model(o2_new, 'orange', new_model_dict)
new_model_dict = new_model(r2_new, 'red', new_model_dict)
new_model_dict = new_model(gb2_new, 'green_b', new_model_dict)
new_model_dict = new_model(gc2_new, 'green_c', new_model_dict)
new_model_dict = new_model(gd2_new, 'green_d', new_model_dict)
new_model_dict = new_model(ge2_new, 'green_e', new_model_dict)


In [83]:
stats = []
p0 = vector.unit(state_space, '0')
for t in trains:
    path = nx.shortest_path(model_dict[t], source="1", target="12")

    vecs = new_model_dict[t].traverse(path, p0)
    means = []
    stds = []
    for v in vecs:
        average = np.average(state_space.values, weights=v)
        # Fast and numerically precise:
        variance = np.average((state_space.values-average)**2, weights=v)
        std = math.sqrt(variance)
        means.append(average)
        stds.append(std)
    df = pd.DataFrame(zip(means, stds), columns=['mean', 'std']).reset_index()
    df.columns = ['element', 'mean', 'std']
    df['train'] = t
    stats.append(df)
new_allstats = pd.concat(stats)
new_allstats
all_allstats = pd.merge(allstats, new_allstats, on=['element', 'train'])
all_allstats.columns = ['element', 'Mean', 'Standard Deviation', 'train', 'New Mean', 'New Standard Deviation']
all_allstats

,element,Mean,Standard Deviation,train,New Mean,New Standard Deviation
0,0,0.000000,0.000000,green_b,0.000000,0.000000
1,1,0.000000,0.000000,green_b,0.000000,0.000000
2,2,18.143082,13.326409,green_b,18.143082,13.326409
3,3,18.143082,13.326409,green_b,18.143082,13.326409
4,4,28.239184,31.408107,green_b,27.389051,30.043674
...,...,...,...,...,...,...
133,18,55.964640,45.246477,green_e,44.403117,36.658612
134,19,55.964640,45.246477,green_e,44.403117,36.658612
135,20,57.119004,45.482102,green_e,46.918793,39.110869
136,21,57.119004,45.482102,green_e,46.918793,39.110869


In [219]:
plot_means = all_allstats[['element', 'train', 'Mean', 'New Mean']].copy()
plot_means.columns = ['element', 'train', 'Before', 'After']
plot_means = plot_means.melt(id_vars=['element', 'train'])
alt.Chart(plot_means).mark_line().encode(
    alt.X('element:O', title='Step'),
    alt.Y('value:Q', title='Schedule Deviation (seconds)'),
    color=alt.Color('train:N', title='Train'),
    strokeDash = alt.StrokeDash('variable:N', title='Scenario', sort='-x')
).properties(
    width = 800,
    height=400,
    title='Modelled Mean Schedule Deviaition Before and After Orange Line Shutdown, Early Peak'
).configure(font='Raleway').configure_view(strokeWidth=0).configure_axis(grid=False, domain=False)

alt.Chart(...)

Afternoon Peak Model

In [85]:
o4_new = o4[0:11]
r4_new = r4[0:11]
gb4_new = gb4[0:11]
gc4_new = gc4[0:11]
gd4_new = gd4[0:11]
ge4_new = ge4[0:11]

In [ ]:
o4_new['from'] = f_num
r4_new['from'] = f_num
gb4_new['from'] = f_num
gc4_new['from'] = f_num
gd4_new['from'] = f_num
ge4_new['from'] = f_num

o4_new['to'] = t_num
r4_new['to'] = t_num
gb4_new['to'] = t_num
gc4_new['to'] = t_num
gd4_new['to'] = t_num
ge4_new['to'] = t_num

In [87]:
model_dict = {}
model_dict = model(o4_new, 'orange', model_dict)
model_dict = model(r4_new, 'red', model_dict)
model_dict = model(gb4_new, 'green_b', model_dict)
model_dict = model(gc4_new, 'green_c', model_dict)
model_dict = model(gd4_new, 'green_d', model_dict)
model_dict = model(ge4_new, 'green_e', model_dict)


In [88]:
stats = []
state_space = StateSpace.from_range(0, 240)
p0 = vector.unit(state_space, '0')
trains = ['green_b','green_c','orange','red','green_d', 'green_e' ]
for t in trains:
    path = nx.shortest_path(model_dict[t], source="1", target="12")
    
    vecs = model_dict[t].traverse(path, p0)
    means = []
    stds = []
    for v in vecs:
        average = np.average(state_space.values, weights=v)
        # Fast and numerically precise:
        variance = np.average((state_space.values-average)**2, weights=v)
        std = math.sqrt(variance)
        means.append(average)
        stds.append(std)
    df = pd.DataFrame(zip(means, stds), columns=['mean', 'std']).reset_index()
    df.columns = ['element', 'mean', 'std']
    df['train'] = t
    stats.append(df)
allstats = pd.concat(stats)
allstats

,element,mean,std,train
0,0,0.000000,0.000000,green_b
1,1,0.000000,0.000000,green_b
2,2,18.143082,13.326409,green_b
3,3,18.143082,13.326409,green_b
4,4,28.239184,31.408107,green_b
...,...,...,...,...
18,18,55.964640,45.246477,green_e
19,19,55.964640,45.246477,green_e
20,20,57.119004,45.482102,green_e
21,21,57.119004,45.482102,green_e


In [89]:
alt.Chart(allstats).mark_line().encode(
    alt.X('element:O', title='Step'),
    alt.Y('mean:Q', title='Modelled Mean Schedule Deviation'),
    color=alt.Color('train:N', title='Train')
).properties(
    width = 800,
    height=500
).configure(font='Raleway').configure_view(strokeWidth=0).configure_axis(grid=False, domain=False)

alt.Chart(...)

In [ ]:
r4_new.iloc[7,5] = 0.1

In [93]:
new_model_dict = {}
new_model_dict = new_model(o4_new, 'orange', new_model_dict)
new_model_dict = new_model(r4_new, 'red', new_model_dict)
new_model_dict = new_model(gb4_new, 'green_b', new_model_dict)
new_model_dict = new_model(gc4_new, 'green_c', new_model_dict)
new_model_dict = new_model(gd4_new, 'green_d', new_model_dict)
new_model_dict = new_model(ge4_new, 'green_e', new_model_dict)

In [94]:
stats = []
p0 = vector.unit(state_space, '0')
for t in trains:
    path = nx.shortest_path(model_dict[t], source="1", target="12")

    vecs = new_model_dict[t].traverse(path, p0)
    means = []
    stds = []
    for v in vecs:
        average = np.average(state_space.values, weights=v)
        # Fast and numerically precise:
        variance = np.average((state_space.values-average)**2, weights=v)
        std = math.sqrt(variance)
        means.append(average)
        stds.append(std)
    df = pd.DataFrame(zip(means, stds), columns=['mean', 'std']).reset_index()
    df.columns = ['element', 'mean', 'std']
    df['train'] = t
    stats.append(df)
new_allstats = pd.concat(stats)
new_allstats
all_allstats = pd.merge(allstats, new_allstats, on=['element', 'train'])
all_allstats.columns = ['element', 'Mean', 'Standard Deviation', 'train', 'New Mean', 'New Standard Deviation']
all_allstats

,element,Mean,Standard Deviation,train,New Mean,New Standard Deviation
0,0,0.000000,0.000000,green_b,0.000000,0.000000
1,1,0.000000,0.000000,green_b,0.000000,0.000000
2,2,18.143082,13.326409,green_b,18.143082,13.326409
3,3,18.143082,13.326409,green_b,18.143082,13.326409
4,4,28.239184,31.408107,green_b,27.389051,30.043674
...,...,...,...,...,...,...
133,18,55.964640,45.246477,green_e,44.403117,36.658612
134,19,55.964640,45.246477,green_e,44.403117,36.658612
135,20,57.119004,45.482102,green_e,46.918793,39.110869
136,21,57.119004,45.482102,green_e,46.918793,39.110869


In [220]:
plot_means = all_allstats[['element', 'train', 'Mean', 'New Mean']].copy()
plot_means.columns = ['element', 'train', 'Before', 'After']
plot_means = plot_means.melt(id_vars=['element', 'train'])
alt.Chart(plot_means).mark_line().encode(
    alt.X('element:O', title='Step'),
    alt.Y('value:Q', title='Schedule Deviation (Seconds)'),
    color=alt.Color('train:N', title='Train'),
    strokeDash = alt.StrokeDash('variable:N', title='Scenario', sort='-x')
).properties(
    width = 800,
    height=400,
    title='Modelled Mean Schedule Deviaition Before and After Orange Line Shutdown, Afternoon Peak'
).configure(font='Raleway').configure_view(strokeWidth=0).configure_axis(grid=False, domain=False)

alt.Chart(...)

Midday Model

In [231]:
o3_new = o3[0:11]
r3_new = r3[0:11]
gb3_new = gb3[0:11]
gc3_new = gc3[0:11]
gd3_new = gd3[0:11]
ge3_new = ge3[0:11]

In [224]:
r3.iloc[7,5] = r3.iloc[7,3] 

In [235]:
model_dict = {}
model_dict = model(o3_new, 'orange', model_dict)
model_dict = model(r3_new, 'red', model_dict)
model_dict = model(gb3_new, 'green_b', model_dict)
model_dict = model(gc3_new, 'green_c', model_dict)
model_dict = model(gd3_new, 'green_d', model_dict)
model_dict = model(ge3_new, 'green_e', model_dict)

In [236]:
stats = []
state_space = StateSpace.from_range(0, 240)
p0 = vector.unit(state_space, '0')
trains = ['green_b','green_c','orange','red','green_d', 'green_e' ]
for t in trains:
    path = nx.shortest_path(model_dict[t], source="1", target="12")
    
    vecs = model_dict[t].traverse(path, p0)
    means = []
    stds = []
    for v in vecs:
        average = np.average(state_space.values, weights=v)
        # Fast and numerically precise:
        variance = np.average((state_space.values-average)**2, weights=v)
        std = math.sqrt(variance)
        means.append(average)
        stds.append(std)
    df = pd.DataFrame(zip(means, stds), columns=['mean', 'std']).reset_index()
    df.columns = ['element', 'mean', 'std']
    df['train'] = t
    stats.append(df)
allstats = pd.concat(stats)
allstats

,element,mean,std,train
0,0,0.000000,0.000000,green_b
1,1,0.000000,0.000000,green_b
2,2,18.143082,13.326409,green_b
3,3,18.143082,13.326409,green_b
4,4,28.239184,31.408107,green_b
...,...,...,...,...
18,18,55.964640,45.246477,green_e
19,19,55.964640,45.246477,green_e
20,20,57.119004,45.482102,green_e
21,21,57.119004,45.482102,green_e


In [237]:
alt.Chart(allstats).mark_line().encode(
    alt.X('element:O', title='Step'),
    alt.Y('mean:Q', title='Modelled Mean Schedule Deviation'),
    color=alt.Color('train:N', title='Train')
).properties(
    width = 800,
    height=500
).configure(font='Raleway').configure_view(strokeWidth=0).configure_axis(grid=False, domain=False)

alt.Chart(...)

In [238]:
new_model_dict = {}
new_model_dict = new_model(o4_new, 'orange', new_model_dict)
new_model_dict = new_model(r4_new, 'red', new_model_dict)
new_model_dict = new_model(gb4_new, 'green_b', new_model_dict)
new_model_dict = new_model(gc4_new, 'green_c', new_model_dict)
new_model_dict = new_model(gd4_new, 'green_d', new_model_dict)
new_model_dict = new_model(ge4_new, 'green_e', new_model_dict)

In [239]:
stats = []
p0 = vector.unit(state_space, '0')
for t in trains:
    path = nx.shortest_path(model_dict[t], source="1", target="12")

    vecs = new_model_dict[t].traverse(path, p0)
    means = []
    stds = []
    for v in vecs:
        average = np.average(state_space.values, weights=v)
        # Fast and numerically precise:
        variance = np.average((state_space.values-average)**2, weights=v)
        std = math.sqrt(variance)
        means.append(average)
        stds.append(std)
    df = pd.DataFrame(zip(means, stds), columns=['mean', 'std']).reset_index()
    df.columns = ['element', 'mean', 'std']
    df['train'] = t
    stats.append(df)
new_allstats = pd.concat(stats)
new_allstats
all_allstats = pd.merge(allstats, new_allstats, on=['element', 'train'])
all_allstats.columns = ['element', 'Mean', 'Standard Deviation', 'train', 'New Mean', 'New Standard Deviation']
all_allstats

,element,Mean,Standard Deviation,train,New Mean,New Standard Deviation
0,0,0.000000,0.000000,green_b,0.000000,0.000000
1,1,0.000000,0.000000,green_b,0.000000,0.000000
2,2,18.143082,13.326409,green_b,18.143082,13.326409
3,3,18.143082,13.326409,green_b,18.143082,13.326409
4,4,28.239184,31.408107,green_b,27.389051,30.043674
...,...,...,...,...,...,...
133,18,55.964640,45.246477,green_e,44.403117,36.658612
134,19,55.964640,45.246477,green_e,44.403117,36.658612
135,20,57.119004,45.482102,green_e,46.918793,39.110869
136,21,57.119004,45.482102,green_e,46.918793,39.110869


In [241]:
plot_means = all_allstats[['element', 'train', 'Mean', 'New Mean']].copy()
plot_means.columns = ['element', 'train', 'Before', 'After']
plot_means = plot_means.melt(id_vars=['element', 'train'])
alt.Chart(plot_means).mark_line().encode(
    alt.X('element:O', title='Step'),
    alt.Y('value:Q', title='Schedule Deviation (Seconds)'),
    color=alt.Color('train:N', title='Train'),
    strokeDash = alt.StrokeDash('variable:N', title='Scenario', sort='-x')
).properties(
    width = 800,
    height=400,
    title='Modelled Mean Schedule Deviaition Before and After Orange Line Shutdown, Midday'
).configure(font='Raleway').configure_view(strokeWidth=0).configure_axis(grid=False, domain=False)

alt.Chart(...)

Path based reliability example

In [165]:
B_501_c['from_stop_name'].unique()

array(['Winship St @ Union St', 'Park St @ Tremont St', nan,
       'Stuart St @ Dartmouth St', 'Lincoln St @ Essex St',
       'Federal St @ Franklin St', 'Chinatown Gate',
       'Saint James Ave @ Dartmouth St', 'Washington St @ Bacon St',
       'Dorchester St @ Dorchester Ave'], dtype=object)

In [ ]:
B_501_c.loc[B_501_c['from_stop_name']=='Park St @ Tremont St']['from_stop_name'] = 'Park Street'
B_501_c.loc[B_501_c['from_stop_name']=='Chinatown Gate']['from_stop_name'] = 'Chinatown'

In [166]:
nodes = ['Haymarket', 'State', 'Downtown Crossing', 'Chinatown', 'Federal St @ Franklin St',
         'Tufts Medical Center','Government Center', 'Park Street', 'Stuart St @ Dartmouth St']

In [189]:
f = ['Haymarket', 'Haymarket', 'State', 'Downtown Crossing', 'Chinatown',
         'Government Center', 'Park Street', 'Park Street','Chinatown','Stuart St @ Dartmouth St',
     'Haymarket']

In [190]:
to = ['State', 'Government Center', 'Downtown Crossing', 'Chinatown', 'Tufts Medical Center',
      'Park Street', 'Downtown Crossing', 'Chinatown', 'Stuart St @ Dartmouth St', 'Tufts Medical Center',
      'Tufts Medical Center']

In [191]:
Map = pd.DataFrame(f, columns = ['from'])
Map['to'] = to

In [192]:
Map

,from,to
0,Haymarket,State
1,Haymarket,Government Center
2,State,Downtown Crossing
3,Downtown Crossing,Chinatown
4,Chinatown,Tufts Medical Center
5,Government Center,Park Street
6,Park Street,Downtown Crossing
7,Park Street,Chinatown
8,Chinatown,Stuart St @ Dartmouth St
9,Stuart St @ Dartmouth St,Tufts Medical Center


In [242]:
Map['mean'] = 0
Map['std'] = 0

Map = fill_mean(Map,Closed,2)
Map = fill_std(Map,Closed,3)

In [243]:
Map.iloc[2,3] = 0.1
Map.iloc[2,2] = 420
Map.iloc[3,3] = 0.1
Map.iloc[3,2] = 360
Map.iloc[4,3] = 0.1
Map.iloc[4,2] = 300
Map.iloc[7,3] = math.sqrt(B_501_c.loc[B_501_c['from_stop_name']=='Park St @ Tremont St']['deviation'].mean())
Map.iloc[7,2] = 420
Map.iloc[8,3] = math.sqrt(B_501_c.loc[B_501_c['from_stop_name']=='Chinatown Gate']['deviation'].mean())
Map.iloc[8,2] = 660
Map.iloc[9,3] = math.sqrt(B_501_c.loc[B_501_c['from_stop_name']=='Stuart St @ Dartmouth St']['deviation'].mean())
Map.iloc[9,2] = 660
Map.iloc[10,3] = 0.1
Map.iloc[10,2] = 1320
Map.iloc[10,3] = 0.1

In [244]:
Map

,from,to,mean,std
0,Haymarket,State,42.208791,4.684529
1,Haymarket,Government Center,75.596578,29.896232
2,State,Downtown Crossing,420.000000,0.100000
3,Downtown Crossing,Chinatown,360.000000,0.100000
4,Chinatown,Tufts Medical Center,300.000000,0.100000
5,Government Center,Park Street,108.017334,38.237078
6,Park Street,Downtown Crossing,38.795625,4.822873
7,Park Street,Chinatown,420.000000,21.235095
8,Chinatown,Stuart St @ Dartmouth St,660.000000,28.741969
9,Stuart St @ Dartmouth St,Tufts Medical Center,660.000000,30.704791


In [215]:
edges = Map
ss = StateSpace.from_range(0, 300)
I = matrix.identity(ss)
N = Network(ss)
for node in nodes:
    N.add_node(node, I)

for idx, edge in edges.iterrows():
    mx = matrix.truncnorm(ss, mean=edge['mean'], std=edge['std'])
    print(edge['from'])
    N.add_edge(edge['from'], edge['to'], mx, data={
        'tt': edge['mean'],
        'ttsd': edge['mean'] + 1.5*edge['std'],
        'tt2sd': edge['mean'] + 2*edge['std']
    })


Haymarket
Haymarket
State
Downtown Crossing
Chinatown
Government Center
Park Street
Park Street
Chinatown
Stuart St @ Dartmouth St
Haymarket


In [245]:
path = nx.shortest_path(N, source='Park Street', target='Tufts Medical Center', weight='tt')
print(path)
length = nx.shortest_path_length(N, source='Park Street', target='Tufts Medical Center', weight='tt')
print(length)

['Park Street', 'Downtown Crossing', 'Chinatown', 'Tufts Medical Center']
698.7956252761821


In [246]:
path = nx.shortest_path(N, source='Park Street', target='Tufts Medical Center', weight='ttsd')
path

['Park Street', 'Downtown Crossing', 'Chinatown', 'Tufts Medical Center']

In [247]:
path = nx.shortest_path(N, source='Park Street', target='Tufts Medical Center', weight='tt2sd')
path

['Park Street', 'Downtown Crossing', 'Chinatown', 'Tufts Medical Center']

In [ ]:
paths = nx.all_simple_paths(N, source='Haymarket', target='Tufts Medical Center')
for path in paths:
  print(path)
  vecs = N.traverse

In [212]:
paths = nx.all_simple_paths(N, source='Haymarket', target='Tufts Medical Center')
p0 = vector.unit(ss, '0')
stats = []
thres = 0.95
for path in paths:
    print(path)
    sm = 0
    vecs = N.traverse(path, p0)
    average = np.average(ss.values, weights=vecs[-1])
    # Fast and numerically precise:
    std = math.sqrt(np.average((ss.values-average)**2, weights=vecs[-1]))
    for i in range(len(vecs[-1])):
        if sm >= thres:
            top = ss.values[i]
            break
        sm += vecs[-1][i]
    stats.append([''.join(path), round(average,2), round(std, 2), top])
df = pd.DataFrame(stats, columns=['Path', 'Mean', 'Std', 'p95'])
df.sort_values(by='Mean').style.hide_index()

['Haymarket', 'State', 'Downtown Crossing', 'Chinatown', 'Tufts Medical Center']


NameError: ignored